In [2]:
!pip install langchain_groq pypdf chromadb sentence_transformers langchain_community streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.5 MB/s eta 0:00:00

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0, #'0' for factual responses. Use 0.7 or 1 for creative responses
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("What is mental wellness? give in 2 lines in english and in german")
print(result.content)

Mental wellness refers to a state of emotional and psychological well-being, where an individual can cope with life's challenges and maintain a positive outlook. It encompasses a range of factors, including self-awareness, resilience, and social connections.

Mentales Wohlbefinden bezeichnet einen Zustand des emotionalen und psychischen Wohlbefindens, in dem eine Person mit den Herausforderungen des Lebens umgehen und eine positive Einstellung bewahren kann. Es umfasst eine Reihe von Faktoren, einschließlich Selbstbewusstsein, Widerstandsfähigkeit und sozialen Verbindungen.


In [3]:
code = """
# === Import Required Libraries ===
import streamlit as st
import os
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

# === UI Setup: Header, Subtitle, and Disclaimer ===
st.markdown(\"\"\"
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Quicksand:wght@600&display=swap');

    .fade-title {
        font-family: 'Quicksand', sans-serif;
        font-size: 42px;
        font-weight: 600;
        color: #333;
        text-align: center;
        margin-top: 20px;
        opacity: 0;
        animation: fadeIn 2s ease-in-out forwards;
    }

    @keyframes fadeIn {
        0% { opacity: 0; transform: translateY(-10px); }
        100% { opacity: 1; transform: translateY(0); }
    }

    .subtitle {
        font-size: 16px;
        text-align: center;
        color: #666;
        margin-bottom: 10px;
        font-family: 'Quicksand', sans-serif;
    }

    .postscript {
        font-size: 13px;
        text-align: center;
        color: #999;
        font-style: italic;
        margin-bottom: 30px;
        font-family: 'Quicksand', sans-serif;
    }
    </style>
\"\"\", unsafe_allow_html=True)

st.markdown('<div class="fade-title">Joy | Talk to Me Anytime</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Your friendly mental wellness companion 💛</div>', unsafe_allow_html=True)
st.markdown('<div class="postscript">This compassionate chatbot provides general support. Please contact a licensed professional for serious concerns.</div>', unsafe_allow_html=True)

# === Custom CSS for Chat Layout ===
st.markdown('''
    <style>
    .chat-container {
        max-height: 70vh;
        overflow-y: auto;
        padding-right: 10px;
        margin-bottom: 80px;
    }

    .chat-input {
        position: fixed;
        bottom: 0;
        left: 0;
        width: 100%;
        background: white;
        padding: 10px 20px;
        z-index: 100;
        box-shadow: 0 -2px 10px rgba(0,0,0,0.1);
    }

    .chatbot-avatar-floating {
        position: fixed;
        bottom: 30px;
        left: 260px;
        width: 200px;
        border-radius: 20px;
        z-index: 1000;
    }

    .block-container {
        padding-bottom: 140px !important;
    }
    </style>
''', unsafe_allow_html=True)

# === 1. Initialize the Language Model (LLM) ===
def initialize_llm():
    return ChatGroq(
        temperature=0.7,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile",
        max_tokens=150
    )

# === 2. Load and Process PDF Documents into Vector DB ===
def create_vector_db():
    loader = DirectoryLoader("/content/Data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    persist_directory = "/tmp/chroma_db"
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
    vector_db.persist()
    return vector_db

# === 3. Setup Question-Answering Chain ===
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = \"\"\"You are a compassionate mental health chatbot. Respond thoughtfully and concisely to the following question:
{context}
User: {question}
Chatbot: (Please keep the answer brief, around 3-4 lines.)\"\"\"

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

# === 4. Load LLM and Vector DB (with caching) ===
@st.cache_resource
def load_components():
    llm = initialize_llm()
    persist_dir = "/tmp/chroma_db"
    if not os.path.exists(persist_dir):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    qa_chain = setup_qa_chain(vector_db, llm)
    return qa_chain

# === 5. Lazy Component Initialization with Loading Animation ===
if "components_ready" not in st.session_state:
    st.session_state.components_ready = False

if not st.session_state.components_ready:
    st.markdown(\"\"\"
        <div style="display: flex; flex-direction: column; align-items: center; justify-content: center; height: 80vh;">
            <img src="https://i.postimg.cc/G24f5p7F/368161870412423172-unscreen.gif" width="140" />
            <p style="margin-top: 20px; font-size: 18px; color: #444;">Joy is getting ready for you...</p>
        </div>
    \"\"\", unsafe_allow_html=True)

    with st.spinner("Loading..."):
        qa_chain = load_components()
        st.session_state.qa_chain = qa_chain
        st.session_state.components_ready = True
    st.rerun()
else:
    qa_chain = st.session_state.qa_chain

# === 6. Floating Chatbot Avatar ===
st.markdown(
    '''
    <img class="chatbot-avatar-floating" src="https://i.postimg.cc/QMJMrxQZ/368133483102908418-unscreen.gif" />
    ''',
    unsafe_allow_html=True
)

# === 7. Initialize Chat History ===
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# === 8. Display Chat Messages ===
st.markdown('<div class="chat-container">', unsafe_allow_html=True)

for speaker, text in st.session_state.chat_history:
    color = "#ffe6f0" if speaker == "You" else "#d0ebff"
    st.markdown(
        f'''
        <div style="background-color: {color}; padding: 10px; border-radius: 10px; margin-bottom: 10px;">
            <strong>{speaker}:</strong><br>{text}
        </div>
        ''',
        unsafe_allow_html=True
    )

st.markdown('</div>', unsafe_allow_html=True)

# === 9. Chat Input Box at Bottom ===
st.markdown('<div class="chat-input">', unsafe_allow_html=True)
user_input = st.chat_input("Ask me anything about mental wellness...")
st.markdown('</div>', unsafe_allow_html=True)

# === 10. Handle User Input and Append Responses ===
if user_input:
    with st.spinner("Thinking..."):
        response = qa_chain.run(user_input)
        st.session_state.chat_history.append(("You", user_input))
        st.session_state.chat_history.append(("Bot", response))
    st.rerun()
"""


with open("app.py", "w") as f:
    f.write(code)


In [ ]:

# Import ngrok from pyngrok
from pyngrok import ngrok

# Set your ngrok authentication token
ngrok.set_auth_token("2vUkcFRJnGeMrIC8Gux1Gmnyx7f_3VVBoUWCy5nZ4roKv1F9x")

# Open a tunnel to the Streamlit app running on localhost:8501
public_url = ngrok.connect("http://localhost:8501")

# Display the public URL where the app will be accessible
print("🔗 Streamlit is live at:", public_url)

# Run the app
!streamlit run app.py &

🔗 Streamlit is live at: NgrokTunnel: "https://3f43-34-23-36-212.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.36.212:8501

/content/app.py:5: LangChainDeprecationWarning: Importing HuggingFaceBgeEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceBgeEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceBgeEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceBgeEmbeddings
/content/app.py:6: LangChainDeprecationWarning: Importing PyPDFLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders i